In [1]:
import re
import numpy as np
import sympy 

# Generates dict of elements for each compound 
# Ex: NaOH becomes {'Na': 1, 'O': 1, 'H': 1} 
def element_dict_for_compound(compound):
    assert "(" not in compound, "This function does not parse parantheses" 
    # Identifies an element along with its number 
    # ? as second letter optional; parantheses as we want the element separate from the number; * because 
    # having no number is ok -- it means 1 
    compound_regex = re.compile('([A-Z][a-z]?)([0-9]*)') 
    element_dict = {element: (int(count) if count else 1) for element, count in compound_regex.findall(compound)}
    return element_dict 

def parse_output(coeffs, inputs): 
        output = "" 
        for index, compound in enumerate(inputs): 
            if index != 0: 
                output += " + " 
            output += str(coeffs[index]) + " " + compound 
        return output 

def main(): 
    # Get a parsed list of reactant compounds 
    print("\nEnter the reactant compounds separated by spaces:")
    reactant_inputs = input().split() 
    parsed_reactant_compounds = [element_dict_for_compound(compound) for compound in reactant_inputs]

    # Get a parsed list of reactant compounds 
    print("\nEnter the product compounds separated by spaces:")
    product_inputs = input().split()
    parsed_product_compounds = [element_dict_for_compound(compound) for compound in product_inputs]

    # Get a list of all the elements in the reaction 
    elements_list = list(set().union(*parsed_reactant_compounds)) ## Make it a list to allow for indexing. 

    m = len(elements_list) 
    n1 = len(reactant_inputs) 
    n2 = len(product_inputs) 
    A1 = np.zeros((m, n1)) 
    A2 = np.zeros((m, n2)) 

    for column, compound in enumerate(parsed_reactant_compounds): 
        for element, count in compound.items(): 
            row = elements_list.index(element) 
            A1[row, column] = count 

    for column, compound in enumerate(parsed_product_compounds): 
        for element, count in compound.items(): 
            row = elements_list.index(element) 
            A2[row, column] = -count 

    A = np.concatenate((A1, A2), axis = 1) 
    A = A.astype(int)

    # Solve using Sympy for absolute-precision math
    # find first basis vector == primary solution
    # find least common denominator, multiply through to convert to integer solution

    A = sympy.Matrix(A)    
    coeffs = A.nullspace()[0]    
    coeffs *= sympy.lcm([term.q for term in coeffs])

    # Display result

    reactant_coeffs = coeffs[:n1] 
    product_coeffs = coeffs[n1:]

    reactant_output = parse_output(reactant_coeffs, reactant_inputs) 
    product_output = parse_output(product_coeffs, product_inputs) 
    final_output = reactant_output + " -> " + product_output

    print("\nBalanced equation:")
    print(final_output) 


if __name__ == "__main__":
    main()

# Ex: 
# NaOH H2SO4
# Na2SO4 H2O 
# Sol: 2 1 1 2 


Enter the reactant compounds separated by spaces:
NaOH H2SO4

Enter the product compounds separated by spaces:
Na2SO4 H2O

Balanced equation:
2 NaOH + 1 H2SO4 -> 1 Na2SO4 + 2 H2O
